In [2]:
import gwaslab as gl
import pandas as pd


In [4]:
# Load gzipped GWAS summary stats
df = pd.read_csv(
    "/mnt/hdd_1/ofgeha/galaxy-gwas-tools/Data/21001_raw.gwas.imputed_v3.both_sexes.tsv.bgz",
    sep="\t",
    compression="gzip"
)

# Split variant into CHR, POS, REF, ALT
df[['CHR', 'POS', 'REF', 'ALT']] = df['variant'].str.split(":", expand=True)
df['POS'] = df['POS'].astype(int)


In [8]:
import gzip
# Replace with your actual path
file_path = "/mnt/hdd_1/ofgeha/galaxy-gwas-tools/Data/21001_raw.gwas.imputed_v3.both_sexes.tsv.bgz"

with gzip.open(file_path, 'rt') as f:
    header = f.readline()
    first_row = f.readline()
    print(f"Header: {repr(header)}")
    print(f"First row: {repr(first_row)}")

Header: 'variant\tminor_allele\tminor_AF\tlow_confidence_variant\tn_complete_samples\tAC\tytx\tbeta\tse\ttstat\tpval\n'
First row: '1:15791:C:T\tT\t5.44688e-09\ttrue\t359983\t3.92157e-03\t1.23895e-01\t8.94616e+02\t1.20487e+03\t7.42499e-01\t4.57786e-01\n'


In [12]:
mysumstats = gl.Sumstats(df,
             snpid="SNP",
             chrom="CHR",
             pos="POS",
             ea="ALT",
             nea="REF",
             neaf="Frq",
             beta="BETA",
             se="SE",
             p="P",
             nrows=5000,
             verbose=False)
mysumstats.basic_check(verbose=False)

KeyError: 'EAF'

In [11]:
mysumstats.data

,NEA,STATUS,EAF,BETA,SE,P,SNPID
0,1:15791:C:T,9999999,1.000000,894.616000,1204.870000,0.457786,<NA>
1,1:69487:G:A,9999999,0.999994,-2.715450,2.360060,0.249902,<NA>
2,1:69569:T:C,9999999,0.999812,-0.484284,0.423462,0.252778,<NA>
3,1:139853:C:T,9999999,0.999994,-2.703560,2.360130,0.251997,<NA>
4,1:692794:CA:C,9999999,0.889410,-0.016436,0.019585,0.401342,<NA>
...,...,...,...,...,...,...,...
13791462,X:154929412:C:T,9999999,0.754527,-0.016260,0.010723,0.129427,<NA>
13791463,X:154929637:CT:C,9999999,0.770274,-0.027098,0.011190,0.015456,<NA>
13791464,X:154929952:CAA:C,9999999,0.760570,-0.020494,0.011278,0.069202,<NA>
13791465,X:154930230:A:G,9999999,0.754113,-0.016347,0.010721,0.127334,<NA>
